Import stuff

In [1]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np

#pv.set_jupyter_backend("ipyvtklink")

# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

Import Envelope Lattice

In [2]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/meshes/voxelized_envelope.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)
envelope_lattice.shape

env_cens = envelope_lattice.centroids_threshold(0)

print (env_cens)

[[-37.8 -92.4   9.6]
 [-37.8 -92.4  19.2]
 [-37.8 -92.4  28.8]
 ...
 [109.2 -16.8   9.6]
 [109.2 -16.8  19.2]
 [109.2 -16.8  28.8]]


Calculate the border points (this calculates the border voxels of the envelope, not the points on the street)

In [3]:
border_lattice = envelope_lattice * 0.0
# border_lattice[:,:,1:-1] += 1.0
border_lattice[:,1:-1,:] += 1.0
border_lattice[1:-1,:,:] += 1.0
border_lattice = border_lattice != 2
border_lattice.centroids

cloud([[ -37.8, -100.8,    0. ],
       [ -37.8, -100.8,    9.6],
       [ -37.8, -100.8,   19.2],
       ...,
       [ 109.2,  -16.8,   19.2],
       [ 109.2,  -16.8,   28.8],
       [ 109.2,  -16.8,   38.4]])

Plotting the code above

In [4]:
p = pv.Plotter(notebook=True)

base_lattice = border_lattice

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the availability lattice
envelope_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")


# Add the data values to the cell data
grid.cell_arrays["Agents"] = base_lattice.flatten(order="F").astype(int)  # Flatten the array!
# filtering the voxels
threshed = grid.threshold([0.9, 1.1])
# adding the voxels
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)


#plotting
cpos = [(314.348192231551, 283.381630231551, 296.74455709155103),
 (65.08283250000001, -12.333572500000002, 21.07374465),
 (0.0, 0.0, 1.0)]
p.camera_position = cpos
p.window_size = 2000, 1000
p.show(use_ipyvtk=True)
p.screenshot("../data/images/streetpoint_creation1")
print(p.camera_position)

ViewInteractiveWidget(height=1000, layout=Layout(height='auto', width='100%'), width=2000)

[(314.348192231551, 283.381630231551, 296.74455709155103),
 (65.08283250000001, -12.333572500000002, 21.07374465),
 (0.0, 0.0, 1.0)]


Calculating the points on the street (defect)

In [5]:
#calculating maximum and minimum values

#minimum values for x and y
xmin, ymin, zmin = env_cens.min(axis=0)

#maximum values for x and y
xmax, ymax, zmax = env_cens.max(axis=0)

#change bound
xmin = xmin - 10
ymin = ymin - 10
xmax = xmax + 10
ymax = ymax + 10

#calculating the points on the street

#calculate the x coördinates
x = np.arange(xmin, xmax)

#calculate the y coördinates
y = np.arange(ymin, ymax)

X = np.tile(x, (2, 1))
X = np.append(X, np.tile(xmin, (len(y), 1)))
X = np.append(X, np.tile(xmax, (len(y), 1)))

Y = np.tile(ymin, (len(x), 1))
Y = np.append(Y, np.tile(ymax, (len(x), 1)))
Y = np.append(Y, np.tile(y, (2, 1)))

Z = np.zeros(len(X))

street_points = np.stack([X,Y,Z]).T
street_points = tg.cloud(street_points)
street_points = street_points.astype(int)
street_points


cloud([[ -47, -106,    0],
       [ -46, -106,    0],
       [ -45, -106,    0],
       ...,
       [ 119,   -9,    0],
       [ 119,   -8,    0],
       [ 119,   -7,    0]])

Visualizing

In [6]:
p = pv.Plotter(notebook=True)

street_points.fast_notebook_vis(p)

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the availability lattice
envelope_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")


# Add the data values to the cell data
grid.cell_arrays["Agents"] = base_lattice.flatten(order="F").astype(int)  # Flatten the array!
# filtering the voxels
threshed = grid.threshold([0.9, 1.1])
# adding the voxels
#p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)


#plotting
cpos = [(314.348192231551, 283.381630231551, 296.74455709155103),
 (65.08283250000001, -12.333572500000002, 21.07374465),
 (0.0, 0.0, 1.0)]
p.camera_position = cpos
p.window_size = 2000, 1000
p.show(use_ipyvtk=True)
p.screenshot("../data/images/streetpoint_creation2")
print(p.camera_position)

ViewInteractiveWidget(height=1000, layout=Layout(height='auto', width='100%'), width=2000)

[(314.348192231551, 283.381630231551, 296.74455709155103),
 (65.08283250000001, -12.333572500000002, 21.07374465),
 (0.0, 0.0, 1.0)]


Saving the streetpoints

In [7]:
street_points_path = os.path.relpath('../data/meshes/street_points.csv')
street_points.to_csv(street_points_path)